In [ ]:
import pandas as pd
import numpy as np
from pycaret.regression import setup, compare_models, blend_models,tune_model,predict_model,get_config, finalize_model

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'Gulim'

In [ ]:
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
from workalendar.asia import SouthKorea
import pendulum

In [ ]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [ ]:
train['중식계']

## 전처리
- 일자에서 월과 일을 분리
- 요일을 레이블 인코딩화(EDA로 요일의 중요도 순 파악)
- 월 별, 일 별 중식 석식 수요 차이 파악

In [ ]:
train['월'] = pd.DatetimeIndex(train['일자']).month.astype(int)
test['월'] = pd.DatetimeIndex(test['일자']).month.astype(int)
train['주'] = pd.DatetimeIndex(train['일자']).week.astype(int)
test['주'] = pd.DatetimeIndex(test['일자']).week.astype(int)
train['일'] = pd.DatetimeIndex(train['일자']).day.astype(int)
test['일'] = pd.DatetimeIndex(test['일자']).day.astype(int)

train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

## 공휴일 변수 생성

In [ ]:
def is_holiday(date):
    holidays = list(map(str, pd.Series(np.array(SouthKorea().holidays(int(date[:4])))[:, 0])))
    
    yesterday = str(np.datetime64(date) - 1)
    tomorrow = str(np.datetime64(date) + 1)

    if tomorrow in holidays and yesterday in holidays:
        return 3 #'S'
    if tomorrow in holidays:
        return 2 # 'T'
    elif yesterday in holidays:
        return 1 #'Y'
    else : 
        return 0 #'N'

def week_of_month(x):
    dt = pendulum.parse(x)
    
    wom = dt.week_of_month
    if wom < 0:
        wom += 52
    return wom
    

df = pd.concat([train[['본사정원수', '일자']], test[['본사정원수', '일자']]])
df['년월'] = df['일자'].apply(lambda x : x[:7])
df = df[['년월', '본사정원수']].groupby(by=['년월'], as_index=False).mean()

def member_change(date):
    this_month = date[:7]
    last_month = str(np.datetime64(this_month) - 1)
    
    this_month_member = int(df[df['년월'] == this_month]['본사정원수'])
    last_month_member = int(df[df['년월'] == last_month]['본사정원수'])
    
    
    return  this_month_member - last_month_member

train['공휴일전후'] = train['일자'].apply(is_holiday)
test['공휴일전후'] = test['일자'].apply(is_holiday)

train['몇주차'] = train['일자'].apply(week_of_month)
test['몇주차'] = test['일자'].apply(week_of_month)

train = train[train['일자'] > '2016-03']
train['인원변화'] = train['일자'].apply(member_change)
test['인원변화'] = test['일자'].apply(member_change)

In [ ]:
month_rank4dinner = {
    1: 11,
    2: 2,
    3: 1,
    4: 4,
    5: 7,
    6: 6,
    7: 10,
    8: 8,
    9: 5,
    10: 3,
    11: 9,
    12: 12
}
train['월(석식)'] = train['월'].map(month_rank4dinner)
test['월(석식)'] = test['월'].map(month_rank4dinner)

month_rank4lunch = {
    1: 3,
    2: 1,
    3: 2,
    4: 6,
    5: 7,
    6: 8,
    7: 10,
    8: 9,
    9: 5,
    10: 4,
    11: 11,
    12: 12
}
train['월(중식)'] = train['월'].map(month_rank4lunch)
test['월(중식)'] = test['월'].map(month_rank4lunch)

weekday_rank4dinner = {
    '월': 1,
    '화': 2,
    '수': 4,
    '목': 3,
    '금': 5,
}

weekday_rank4lunch = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5,
}

rank = pd.DataFrame(range(1,53))
week_rank_lunch = pd.pivot_table(train,values='중식계',index='주').sort_values(by='중식계').reset_index().drop('중식계',axis=1)
week_rank_dinner = pd.pivot_table(train,values='석식계',index='주').sort_values(by='석식계').reset_index().drop('석식계',axis=1)


week_rank4lunch = {}
for i in range(len(rank)):
    week_rank4lunch[week_rank_lunch['주'][i]] = rank[0][i]


week_rank4dinner = {}
for i in range(len(rank)):
    week_rank4dinner[week_rank_dinner['주'][i]] = rank[0][i]
    
    
train['주(중식)'] = train['주'].map(week_rank4lunch)
test['주(중식)'] = test['주'].map(week_rank4lunch)

train['주(석식)'] = train['주'].map(week_rank4dinner)
test['주(석식)'] = test['주'].map(week_rank4dinner)

train['요일(석식)'] = train['요일'].map(weekday_rank4dinner)
test['요일(석식)'] = test['요일'].map(weekday_rank4dinner)

train['요일(중식)'] = train['요일'].map(weekday_rank4lunch)
test['요일(중식)'] = test['요일'].map(weekday_rank4lunch)

In [ ]:
train.drop(columns=['조식메뉴', '중식메뉴', '석식메뉴'], inplace=True)

## 최종 데이터 셋 구축

In [ ]:
train.columns

In [ ]:
lunch_train = train.drop(columns=['본사정원수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수','일자', '요일','주', '월', '석식계', '요일(석식)','조식메뉴', '중식메뉴', '석식메뉴','주(석식)', '월(석식)'])
lunch_test = test.drop(columns=['본사정원수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수','일자', '요일','주', '월', '요일(석식)', '월(석식)', '조식메뉴', '중식메뉴','주(석식)' , '석식메뉴'])


dinner_train = train.drop(columns=['본사정원수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수','일자', '요일','주', '월', '중식계', '요일(중식)','조식메뉴', '중식메뉴', '석식메뉴','주(중식)' , '월(중식)'])
dinner_test = test.drop(columns=['본사정원수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수','일자', '요일', '주','월', '요일(중식)', '월(중식)', '조식메뉴', '중식메뉴','주(중식)' , '석식메뉴'])

In [ ]:
print(lunch_train.shape)
print(lunch_test.shape)

In [ ]:
print(dinner_train.shape)
print(dinner_test.shape)

In [ ]:
lunch_train.columns

#### 분포 확인 및 분포 조정

In [ ]:
drop_index = dinner_train[dinner_train['석식계']==0].index

dinner_train.iloc[drop_index]

In [ ]:
dinner_train.drop(drop_index, inplace=True)

print(dinner_train.shape)

# 중식 예측모델

In [ ]:
y_lunch = np.array(lunch_train['중식계'])
y_lunch = y_lunch.reshape(-1)
lunch_train.drop(columns=['중식계'], inplace=True)

y_dinner = np.array(dinner_train['석식계'])
y_dinner = y_dinner.reshape(-1)
dinner_train.drop(columns=['석식계'], inplace=True)

In [ ]:
cat_features = [f for f in lunch_train.columns if lunch_train[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(lunch_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

In [ ]:
n_fold = 8

kfold = KFold(n_splits=n_fold, shuffle=False)
# for each fold

lunch_models_cat = [CatBoostRegressor(n_estimators = 3000, loss_function='MAE') for _ in range(n_fold)]
lunch_preds_cat = []

lunch_models_xgb = [XGBRegressor(n_estimators = 3000, loss_function='MAE') for _ in range(n_fold)]
lunch_preds_xgb = []

lunch_models_lgb = [GradientBoostingRegressor() for _ in range(n_fold)]
lunch_preds_lgb = []

lunch_trues = []

for idx, (tr_idx, val_idx) in enumerate(kfold.split(lunch_train, y_lunch)):
    X_tr, X_val = lunch_train.iloc[tr_idx], lunch_train.iloc[val_idx]
    y_tr, y_val = y_lunch[tr_idx], y_lunch[val_idx]

    temp_model_cat = lunch_models_cat[idx]
    temp_model_cat.fit(X_tr, y_tr,
                   eval_set=[(X_tr, y_tr), (X_val, y_val)],
                   early_stopping_rounds=50,
                   verbose=400)
    
    lunch_models_cat[idx] = temp_model_cat
    
    pred_cat = temp_model_cat.predict(X_val)
    lunch_preds_cat.extend(pred_cat)

    temp_model_xgb = lunch_models_xgb[idx]
    temp_model_xgb.fit(X_tr, y_tr,
                   eval_set=[(X_tr, y_tr), (X_val, y_val)],
                   early_stopping_rounds=50,
                   verbose=400)
    
    lunch_models_xgb[idx] = temp_model_xgb
    
    pred_xgb = temp_model_xgb.predict(X_val)
    lunch_preds_xgb.extend(pred_xgb)

    temp_model_lgb = lunch_models_lgb[idx]
    temp_model_lgb.fit(X_tr, y_tr)
#                    eval_set=[(X_tr, y_tr), (X_val, y_val)],
#                    early_stopping_rounds=50,
#                    verbose=400)
    
    lunch_models_lgb[idx] = temp_model_lgb
    
    pred_lgb = temp_model_lgb.predict(X_val)
    
    lunch_preds_lgb.extend(pred_lgb)
    
    true = y_val
    lunch_trues.extend(true)

### 테스트 데이터 예측

In [ ]:
lunch_preds_cat = np.array(lunch_preds_cat)
lunch_preds_xgb = np.array(lunch_preds_xgb)
lunch_preds_lgb = np.array(lunch_preds_lgb)

lunch_trues = np.array(lunch_trues)

abs(lunch_trues-lunch_preds_cat).mean(), abs(lunch_trues-lunch_preds_xgb).mean(), abs(lunch_trues-lunch_preds_lgb).mean()

In [ ]:
lunch = np.array([0]*50).astype(np.float64)

for model in lunch_models_cat:
    lunch += model.predict(lunch_test)
    
for model in lunch_models_xgb:
    lunch += model.predict(lunch_test)
    
for model in lunch_models_lgb:
    lunch += model.predict(lunch_test)
    
lunch /= n_fold * 3

# 석식 예측모델

In [ ]:
dinner_models_cat = [CatBoostRegressor(n_estimators = 3000, loss_function='MAE') for _ in range(n_fold)]
dinner_preds_cat = []

dinner_models_xgb = [XGBRegressor(n_estimators = 3000, loss_function='MAE') for _ in range(n_fold)]
dinner_preds_xgb = []

dinner_models_lgb = [GradientBoostingRegressor() for _ in range(n_fold)]
dinner_preds_lgb = []

dinner_trues = []

for idx, (tr_idx, val_idx) in enumerate(kfold.split(dinner_train, y_dinner)):
    X_tr, X_val = dinner_train.iloc[tr_idx], dinner_train.iloc[val_idx]
    y_tr, y_val = y_dinner[tr_idx], y_dinner[val_idx]

    temp_model_cat = dinner_models_cat[idx]
    temp_model_cat.fit(X_tr, y_tr,
                   eval_set=[(X_tr, y_tr), (X_val, y_val)],
                   early_stopping_rounds=50,
                   verbose=400)
    
    dinner_models_cat[idx] = temp_model_cat
    
    pred_cat = temp_model_cat.predict(X_val)
    dinner_preds_cat.extend(pred_cat)

    temp_model_xgb = dinner_models_xgb[idx]
    temp_model_xgb.fit(X_tr, y_tr,
                   eval_set=[(X_tr, y_tr), (X_val, y_val)],
                   early_stopping_rounds=50,
                   verbose=400)
    
    dinner_models_xgb[idx] = temp_model_xgb
    
    pred_xgb = temp_model_xgb.predict(X_val)
    dinner_preds_xgb.extend(pred_xgb)

    temp_model_lgb = dinner_models_lgb[idx]
    temp_model_lgb.fit(X_tr, y_tr)
#                    eval_set=[(X_tr, y_tr), (X_val, y_val)],
#                    early_stopping_rounds=50,
#                    verbose=400)
    
    dinner_models_lgb[idx] = temp_model_lgb
    
    pred_lgb = temp_model_lgb.predict(X_val)
    
    dinner_preds_lgb.extend(pred_lgb)
    
    true = y_val
    dinner_trues.extend(true)

In [ ]:
dinner_preds_cat = np.array(dinner_preds_cat)
dinner_preds_xgb = np.array(dinner_preds_xgb)
dinner_preds_lgb = np.array(dinner_preds_lgb)

dinner_trues = np.array(dinner_trues)

abs(dinner_trues-dinner_preds_cat).mean(), abs(dinner_trues-dinner_preds_xgb).mean(), abs(dinner_trues-dinner_preds_lgb).mean()

In [ ]:
singles = set()
for idx, model in enumerate(dinner_models_xgb):
    singles.add((f'model_{idx}', model))

In [ ]:
dinner = np.array([0]*50).astype(np.float64)

for model in dinner_models_cat:
    dinner += model.predict(dinner_test)
    
for model in dinner_models_xgb:
    dinner += model.predict(dinner_test)
    
for model in dinner_models_lgb:
    dinner += model.predict(dinner_test)
    
dinner /= n_fold * 3

## 테스트

### 테스트 데이터 예측

In [ ]:
submission = pd.read_csv('../data/sample_submission.csv')
submission.iloc[:,1] = lunch
submission.iloc[:,2] = dinner

submission.head()

In [ ]:
answer = pd.read_csv('../submission/제출해야될것.csv')

lunch_answer = np.array(answer.iloc[:,1])
dinner_answer = np.array(answer.iloc[:,2])

abs(lunch - lunch_answer).mean(), abs(dinner - dinner_answer).mean()

In [ ]:
submission.head(11)

# 저장

In [ ]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_cat_xgb_8.csv', index =False)

In [ ]:
def plot_feature_importance(importance, names, model_type):
    
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    plt.figure(figsize=(10,8))

    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])

    plt.title(model_type + ' Feature Importance')
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature Names')

In [ ]:
for i in range(n_fold):
    plot_feature_importance(dinner_models[0].get_feature_importance(),dinner_train.iloc[:,:-1].columns,"CATBOOST")

In [ ]:
lunch_train